# Chapter 3: State Estimation

## Principles of Indoor Positioning and Indoor Navigation

---

### 📚 Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** different state estimation techniques (LS, WLS, Robust LS, KF, EKF)
2. **Implement** Least Squares estimation for static positioning
3. **Apply** robust estimation to handle measurement outliers
4. **Use** Kalman Filter for dynamic state tracking
5. **Compare** different estimators in terms of accuracy and robustness

### 📖 Book Reference

This notebook covers **Chapter 3: State Estimation** with:
- **Eq. (3.1)-(3.5)**: Ordinary Least Squares (OLS)
- **Eq. (3.6)-(3.10)**: Weighted Least Squares (WLS)
- **Eq. (3.11)-(3.15)**: Robust estimation (Huber, Cauchy loss)
- **Eq. (3.16)-(3.25)**: Kalman Filter
- **Eq. (3.26)-(3.35)**: Extended Kalman Filter (EKF)

---


## 🚀 Setup (Google Colab)

**Set the `GITHUB_REPO` variable below to your repository URL, then run the setup cell.**

Example: `GITHUB_REPO = "https://github.com/YOUR_USERNAME/IPIN_Book_Examples.git"`


In [ ]:
# ========================================
# IPIN Book Examples - Chapter 3: State Estimation
# ========================================

import os
import sys

# ============ CONFIGURATION ============
GITHUB_REPO = None  # Set your repo URL, e.g., "https://github.com/username/IPIN_Book_Examples.git"
# =======================================

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    if os.path.exists('/content/IPIN_Book_Examples/core'):
        os.chdir('/content/IPIN_Book_Examples')
        print("✅ Repository already available.")
    elif GITHUB_REPO:
        print(f"📥 Cloning from {GITHUB_REPO}...")
        get_ipython().system(f'git clone {GITHUB_REPO}')
        os.chdir('/content/IPIN_Book_Examples')
        get_ipython().system('pip install -e . -q')
        print("✅ Setup from GitHub complete!")
    else:
        print("❌ ERROR: GITHUB_REPO not set!")
        print("Please set GITHUB_REPO = 'https://github.com/YOUR_USERNAME/IPIN_Book_Examples.git'")
        raise ValueError("GITHUB_REPO not configured.")
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
    print(f"📂 Working directory: {os.getcwd()}")

# Import libraries
import numpy as np
import matplotlib.pyplot as plt

get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

# Import estimation modules
from core.estimators import KalmanFilter, ExtendedKalmanFilter

print("\n" + "="*60)
print("✅ Setup complete! State estimation modules loaded.")
print("="*60)


---

# Part 1: Least Squares Estimation

## 1.1 Theory

**Linear measurement model**: z = Hx + v

**Least Squares solution** (Eq. 3.3): x = (H'H)⁻¹H'z

**Weighted Least Squares** (Eq. 3.8): x = (H'WH)⁻¹H'Wz


In [ ]:
# ============================================
# Part 1: Least Squares Estimation
# ============================================

np.random.seed(42)

# True position
x_true = np.array([5.0, 3.0])

# Anchor positions (reference stations)
anchors = np.array([
    [0, 0],
    [10, 0],
    [10, 10],
    [0, 10],
])
n_anchors = len(anchors)

# True ranges
ranges_true = np.linalg.norm(anchors - x_true, axis=1)

# Add noise
sigma_range = 0.3  # meters
ranges_meas = ranges_true + sigma_range * np.random.randn(n_anchors)

print("True position:", x_true)
print("True ranges:", ranges_true)
print("Measured ranges:", ranges_meas)


In [ ]:
# ============================================
# Iterative Weighted Least Squares
# ============================================

def solve_position_wls(ranges, anchors, sigma, x_init=None, max_iter=10):
    """Solve position using Weighted Least Squares."""
    if x_init is None:
        x_init = np.mean(anchors, axis=0)
    
    x = x_init.copy()
    W = np.eye(len(ranges)) / sigma**2
    
    for _ in range(max_iter):
        d = anchors - x
        r_pred = np.linalg.norm(d, axis=1)
        H = -d / r_pred[:, np.newaxis]
        y = ranges - r_pred
        HtWH = H.T @ W @ H
        HtWy = H.T @ W @ y
        dx = np.linalg.solve(HtWH, HtWy)
        x = x + dx
        if np.linalg.norm(dx) < 1e-6:
            break
    return x

x_wls = solve_position_wls(ranges_meas, anchors, sigma_range)
error_wls = np.linalg.norm(x_wls - x_true)

print(f"WLS estimated position: {x_wls}")
print(f"Position error: {error_wls:.3f} m")


In [ ]:
# Visualize WLS Result
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(anchors[:, 0], anchors[:, 1], s=200, c='orange', marker='^',
           edgecolors='black', linewidths=2, label='Anchors', zorder=5)
ax.plot(x_true[0], x_true[1], 'g*', markersize=20, label='True Position')
ax.plot(x_wls[0], x_wls[1], 'r^', markersize=15, label=f'WLS Estimate (err={error_wls:.3f}m)')

for i, (anchor, r) in enumerate(zip(anchors, ranges_meas)):
    circle = plt.Circle(anchor, r, fill=False, linestyle='--', alpha=0.5)
    ax.add_patch(circle)

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('Weighted Least Squares Position Estimation')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')
ax.set_xlim(-2, 12)
ax.set_ylim(-2, 12)
plt.tight_layout()
plt.show()


---

# Part 2: Kalman Filter

## 2.1 Theory

**State model**: x(k+1) = F*x(k) + w

**Measurement model**: z(k) = H*x(k) + v

**Prediction**: x⁻ = F*x, P⁻ = F*P*F' + Q

**Update**: K = P⁻*H'*(H*P⁻*H' + R)⁻¹, x = x⁻ + K*(z - H*x⁻)


In [ ]:
# ============================================
# Part 2: Kalman Filter - 1D Tracking
# ============================================

np.random.seed(42)

# Simulation parameters
dt = 0.1  # Time step (s)
n_steps = 100
t = np.arange(n_steps) * dt

# True trajectory (constant velocity with small process noise)
x_true_kf = np.zeros((n_steps, 2))
x_true_kf[0] = [0, 1.0]  # Start at position 0, velocity 1 m/s

sigma_process = 0.1
for i in range(1, n_steps):
    x_true_kf[i, 1] = x_true_kf[i-1, 1] + sigma_process * np.random.randn() * dt
    x_true_kf[i, 0] = x_true_kf[i-1, 0] + x_true_kf[i-1, 1] * dt

# Generate noisy position measurements
sigma_meas = 0.5
z_meas = x_true_kf[:, 0] + sigma_meas * np.random.randn(n_steps)

print(f"Generated {n_steps} measurements over {n_steps*dt:.1f} seconds")


In [ ]:
# Run Kalman Filter
F = np.array([[1, dt], [0, 1]])
H = np.array([[1, 0]])
Q = np.array([[dt**4/4, dt**3/2], [dt**3/2, dt**2]]) * sigma_process**2
R = np.array([[sigma_meas**2]])

x_est = np.array([z_meas[0], 0])
P_est = np.diag([sigma_meas**2, 1.0])

x_kf = np.zeros((n_steps, 2))
P_kf = np.zeros((n_steps, 2, 2))

for i in range(n_steps):
    if i > 0:
        x_est = F @ x_est
        P_est = F @ P_est @ F.T + Q
    
    y = z_meas[i] - H @ x_est
    S = H @ P_est @ H.T + R
    K = P_est @ H.T @ np.linalg.inv(S)
    x_est = x_est + K.flatten() * y
    P_est = (np.eye(2) - K @ H) @ P_est
    
    x_kf[i] = x_est
    P_kf[i] = P_est

error_meas = np.abs(z_meas - x_true_kf[:, 0])
error_kf = np.abs(x_kf[:, 0] - x_true_kf[:, 0])

print(f"Measurement RMSE: {np.sqrt(np.mean(error_meas**2)):.3f} m")
print(f"KF Position RMSE: {np.sqrt(np.mean(error_kf**2)):.3f} m")
print(f"Improvement: {100*(1 - np.sqrt(np.mean(error_kf**2))/np.sqrt(np.mean(error_meas**2))):.1f}%")


In [ ]:
# Visualize Kalman Filter Results
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

ax = axes[0]
ax.plot(t, x_true_kf[:, 0], 'b-', linewidth=2, label='True Position')
ax.scatter(t, z_meas, s=20, c='gray', alpha=0.5, label='Measurements')
ax.plot(t, x_kf[:, 0], 'r-', linewidth=2, label='KF Estimate')
sigma_pos = np.sqrt(P_kf[:, 0, 0])
ax.fill_between(t, x_kf[:, 0] - 2*sigma_pos, x_kf[:, 0] + 2*sigma_pos,
                color='red', alpha=0.2, label='±2σ')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Position (m)')
ax.set_title('Kalman Filter: Position Tracking')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(t, x_true_kf[:, 1], 'b-', linewidth=2, label='True Velocity')
ax.plot(t, x_kf[:, 1], 'r-', linewidth=2, label='KF Estimate')
sigma_vel = np.sqrt(P_kf[:, 1, 1])
ax.fill_between(t, x_kf[:, 1] - 2*sigma_vel, x_kf[:, 1] + 2*sigma_vel,
                color='red', alpha=0.2, label='±2σ')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Velocity (m/s)')
ax.set_title('Kalman Filter: Velocity Estimation (Unobserved State)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


---

# Summary

## Key Takeaways

| Method | Use Case | Key Feature |
|--------|----------|-------------|
| **LS** | Static, overdetermined | Simple, optimal for Gaussian noise |
| **WLS** | Heteroscedastic noise | Accounts for varying measurement quality |
| **KF** | Dynamic systems | Optimal for linear Gaussian systems |
| **EKF** | Nonlinear systems | Linearization around current estimate |

### Key Insights

1. **Kalman Filter** estimates unobserved states (velocity from position only!)
2. **Covariance** quantifies estimation uncertainty
3. **Robust methods** are essential for real-world NLOS environments

---

**Next Steps:** Chapter 4 (RF Positioning) to apply these estimators to real positioning problems!
